In [ ]:
import os
os.environ['PYSPARK_SUBMIT_ARGS'] = '--conf spark.ui.port=4041 --packages org.apache.kafka:kafka_2.11:0.10.0.0,org.apache.kafka:kafka-clients:0.10.0.0  pyspark-shell'


In [ ]:
, json.dumps(job).encode("utf-8")

In [ ]:
!pip install kafka-python

In [1]:
import os
os.environ['PYSPARK_SUBMIT_ARGS'] = '--conf spark.ui.port=4041 --packages org.apache.kafka:kafka_2.11:0.10.0.0,org.apache.kafka:kafka-clients:0.10.0.0  pyspark-shell'

import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
import requests
import re
from kafka import KafkaProducer
import json
import time

topic_name = "seekjobs"
producer = KafkaProducer(bootstrap_servers='localhost:9092')

baseurl = "https://www.seek.com.au/machine-learning-jobs/in-All-Sydney-NSW/full-time"
def extract(baseurl):
    
    headers={'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/106.0.0.0 Safari/537.36'}
    #baseurl = "https://www.seek.com.au/machine-learning-jobs/in-All-Sydney-NSW/full-time?page={start}"
    response = requests.get(baseurl)
    html = response.text
    soup = BeautifulSoup(html, 'html.parser')
    return soup

def pages(soup):
    page=soup.find('ul',class_='yvsb870 yvsb873 _14uh99456 _14uh994em')
    if not page.find('li',{"data-automation":"page-26"}):
        url='https://www.seek.com.au' + str(page.find('li',class_="yvsb870 _14uh9944q _14uh9944v _14uh9948b").find('a')['href'])
        return url
    else:
        print("page over") 

def get_job_summary(soup):
    #yvsb870 yvsb871 h3f08h1 _14uh9946i _14uh9947i _14uh9949m _14uh9948m _14uh9945a
    #yvsb870 yvsb871 h3f08h1 _14uh9946i _14uh9947i _14uh9949m _14uh9948m _14uh9945a h3f08h5
    divs=soup.find_all('article',{'data-automation':"normalJob"})
    for item in divs:
        jobD = item.find('span', {'data-automation':'jobShortDescription'})
        if (jobD is None):
                jobD='NA'
        else:
                jobD= jobD.text
        #print(jobD)
        job_title = item.find('a', {'data-automation':'jobTitle'})
        if (job_title is None):
                job_title='NA'
        else:
                job_title = job_title.text
                
        job_salary = item.find('span', {'data-automation':'jobSalary'})
        if (job_salary is None):
            job_salary = 'NA'
        else:
            job_salary = job_salary.text
         
        job_company = item.find('a', {'data-automation':'jobCompany'})
        if (job_company is None):
            job_company = 'NA'
        else:
            job_company = job_company.text
        #print(job_company)
        
            
        job_loc = item.find('a', {'data-automation':'jobLocation'})
        if (job_loc is None):
            job_loc = 'NA'
        else:
            job_loc = job_loc.text
        
        job_clas = item.find('a', {'data-automation':'jobClassification'})
        if (job_clas is None):
            job_clas = 'NA'
        else:
            job_clas = job_clas.text
            
            
        date=item.find('span',{"data-automation":"jobListingDate"})
        if(date is None):
            date='NA'
        else:
            date=date.text
        job={ 'description':jobD,
            'title':job_title,
            'salary':job_salary,
            'company': job_company,
            'location':job_loc,
            'class':job_clas,
            'days_before':date
            
        }
        
        producer.send(topic_name, json.dumps(job).encode("utf-8"))
        #time.sleep(1)
    return
  
for i in range(1,27):
    #print(f'getting page,{i}')
    soup= extract(baseurl)   
    baseurl=pages(soup)
    if not baseurl:
        break
    #print(len(job_title))
    get_job_summary(soup)       
    


    
